# Mutual Self-Attention + ControlNet
This notebook supports image generation using the Mutual Self-Attention + ControlNet pipeline proposed in the report.

The model that supports ControlNet, `MasaCtrlControlNetPipeline`, is implemented in `diffuser_utils.py`.


The code work as follows:

You first register the `MutualSelfAttention` editor to the `MasaCtrlControlNetPipeline` using the `register_attention_editor_diffusers` function.
Then, if you generate an image using a paired prompt in format ['source prompt', 'edited prompt'] along with a conditioning input image, the output will be saved in the `workdir/exp` directory.


# Install Dependencies

In [7]:
!pip install -r requirements.txt

  Using cached diffusers-0.33.1-py3-none-any.whl.metadata (19 kB)
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached pytorch_lightning-2.5.1.post0-py3-none-any.whl.metadata (20 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached pillow-11.2.1-cp39-cp39-win_amd64.whl.metadata (9.1 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached tokenizers-0.2

In [8]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp39-cp39-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp39-cp39-win_amd64.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp39-cp39-win_amd64.whl.metadata (27 kB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp39-cp39-win_amd64.whl (5.5 MB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp39-cp39-win_amd64.whl (2817.1 MB)
Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp39-cp39-win_amd64.whl (4.1 MB)

  Attempting uninstall: torch

    Found existing installation: torch 2.7.1

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
    Un

### Mutual Self-Attention + ControlNet

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
from einops import rearrange, repeat
from omegaconf import OmegaConf

from diffusers import DDIMScheduler, ControlNetModel

from MasaCtrl.masactrl.diffuser_utils import MasaCtrlPipeline, MasaCtrlControlNetPipeline
from MasaCtrl.masactrl.masactrl_utils import AttentionBase
from MasaCtrl.masactrl.masactrl_utils import regiter_attention_editor_diffusers

from torchvision.utils import save_image
from torchvision.io import read_image
from pytorch_lightning import seed_everything

torch.cuda.set_device(0)  # set the GPU device

#### Model Construction

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_path = "stable-diffusion-v1-5/stable-diffusion-v1-5"
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose")

model = MasaCtrlControlNetPipeline.from_pretrained(model_path, controlnet=controlnet, scheduler=scheduler, cross_attention_kwargs={"scale": 0.5}).to(device)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Keyword arguments {'cross_attention_kwargs': {'scale': 0.5}} are not expected by MasaCtrlControlNetPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 16.75it/s]


#### Consistent synthesis with MasaCtrl

In [4]:
from MasaCtrl.masactrl.masactrl import MutualSelfAttentionControl
from PIL import Image


seed = 42
seed_everything(seed)

out_dir = "./workdir/exp/"
os.makedirs(out_dir, exist_ok=True)
sample_count = len(os.listdir(out_dir))
out_dir = os.path.join(out_dir, f"sample_{sample_count}")
os.makedirs(out_dir, exist_ok=True)

prompts = [
    "1boy, casual, outdoors, standing",  # source prompt
    "1boy, casual, outdoors, dancing"  # target prompt
]

condition_image = "dataset/poses/dance_03.png"
# load the condition image
condition_image = read_image(condition_image).float() / 255.0
# rgba to rgb conversion
if condition_image.shape[0] == 4:
    condition_image = condition_image[:3, :, :]
    # resize to 512x512
condition_image = F.interpolate(condition_image.unsqueeze(0), size=(512, 512), mode='bilinear', align_corners=False)
condition_image = condition_image.to(device)
zero_condition = torch.zeros_like(condition_image)
condition = torch.cat([zero_condition, condition_image], dim=0)  # concatenate the condition image and zero condition

# initialize the noise map
start_code = torch.randn([1, 4, 64, 64], device=device)
start_code = start_code.expand(len(prompts), -1, -1, -1)



Seed set to 42


In [5]:
# inference the synthesized image without MasaCtrl
editor = AttentionBase()
regiter_attention_editor_diffusers(model, editor)
image_ori = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)

c:\Users\hongbin\Desktop\AI618_Final\MasaCtrl\masactrl\diffuser_utils.py:367: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  latent_shape = (batch_size, self.unet.in_channels, height // 8, width // 8)
DDIM Sampler: 100%|██████████| 50/50 [18:26<00:00, 22.13s/it]


In [6]:
# inference the synthesized image with MasaCtrl
STEP = 4
LAYER = 10

# hijack the attention module
editor = MutualSelfAttentionControl(STEP, LAYER)
regiter_attention_editor_diffusers(model, editor)

# inference the synthesized image
image_masactrl = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)[-1:]

# save the synthesized image
out_image = torch.cat([image_ori, image_masactrl], dim=0)
save_image(out_image, os.path.join(out_dir, f"all_step{STEP}_layer{LAYER}.png"))
save_image(out_image[0], os.path.join(out_dir, f"source_step{STEP}_layer{LAYER}.png"))
save_image(out_image[1], os.path.join(out_dir, f"without_step{STEP}_layer{LAYER}.png"))
save_image(out_image[2], os.path.join(out_dir, f"masactrl_step{STEP}_layer{LAYER}.png"))

print("Syntheiszed images are saved in", out_dir)

MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]


DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler: 100%|██████████| 50/50 [20:06<00:00, 24.14s/it]


Syntheiszed images are saved in ./workdir/exp/sample_4


# Batch Generation

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
from einops import rearrange, repeat
from omegaconf import OmegaConf

from diffusers import DDIMScheduler, ControlNetModel

from MasaCtrl.masactrl.diffuser_utils import MasaCtrlPipeline, MasaCtrlControlNetPipeline
from MasaCtrl.masactrl.masactrl_utils import AttentionBase
from MasaCtrl.masactrl.masactrl_utils import regiter_attention_editor_diffusers

from torchvision.utils import save_image
from torchvision.io import read_image
from pytorch_lightning import seed_everything

torch.cuda.set_device(0)  # set the GPU device

/mnt/hdd/hbchoe/anaconda3/envs/Masactrl2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from MasaCtrl.masactrl.masactrl import MutualSelfAttentionControl
from PIL import Image


seed = 42
seed_everything(seed)



Seed set to 45


45

In [128]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_path = "stable-diffusion-v1-5/stable-diffusion-v1-5"
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose")

model = MasaCtrlControlNetPipeline.from_pretrained(model_path, controlnet=controlnet, scheduler=scheduler, cross_attention_kwargs={"scale": 0.5}).to(device)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Keyword arguments {'cross_attention_kwargs': {'scale': 0.5}} are not expected by MasaCtrlControlNetPipeline and will be ignored.
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [129]:
prompts = [
    "highly detailed, 1boy, standing, facing camera, full body portrait, full-length portrait",  # source prompt
    "highly detailed, 1boy, dancing, facing camera, full body portrait, full-length portrait"  # target prompt
]

condition_image = "dataset/poses/dance_01.png"
# load the condition image
condition_image = read_image(condition_image).float() / 255.0
# rgba to rgb conversion
if condition_image.shape[0] == 4:
    condition_image = condition_image[:3, :, :]
    # resize to 512x512
condition_image = F.interpolate(condition_image.unsqueeze(0), size=(512, 512), mode='bilinear', align_corners=False)
condition_image = condition_image.to(device)
zero_condition = torch.zeros_like(condition_image)
condition = torch.cat([zero_condition, condition_image], dim=0)  # concatenate the condition image and zero condition

# initialize the noise map
start_code = torch.randn([1, 4, 64, 64], device=device)
start_code = start_code.expand(len(prompts), -1, -1, -1)


In [130]:
# inference the synthesized image without MasaCtrl
editor = AttentionBase()
regiter_attention_editor_diffusers(model, editor)
image_ori = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)

DDIM Sampler: 100%|██████████| 50/50 [00:53<00:00,  1.07s/it]


In [131]:
from torchvision.transforms import ToPILImage
# Convert the PyTorch tensor to PIL image before saving
ToPILImage()(image_ori[0].cpu()).save("final_test22/final_test_zero_cond_original.png")
ToPILImage()(image_ori[1].cpu()).save("final_test22/final_test_zero_cond_without.png")


In [132]:
import glob
import os
from torchvision.utils import save_image

STEP = 4
LAYER = 10
# sequential generation

folder_path = "/mnt/hdd/hbchoe/workspace/MasaCtrl/dataset/poses"
output_folder = "final_test22"
control_image_files = sorted(glob.glob(f"{folder_path}/*.png"))

# conditioning image preprocess
condition_image = "/mnt/hdd/hbchoe/workspace/MasaCtrl/dataset/poses/dance_03.png"



for file in control_image_files[:5]:
    # load the condition image
    condition_image = read_image(file).float() / 255.0
    # rgba to rgb conversion
    if condition_image.shape[0] == 4:
        condition_image = condition_image[:3, :, :]
        # resize to 512x512
    condition_image = F.interpolate(condition_image.unsqueeze(0), size=(512, 512), mode='bilinear', align_corners=False)
    condition_image = condition_image.to(device)
    zero_condition = torch.zeros_like(condition_image)
    condition = torch.cat([zero_condition, condition_image], dim=0)  # concatenate the condition image and zero condition

    # inference the synthesized image with MasaCtrl
    STEP = 4
    LAYER = 10

    # hijack the attention module
    # editor = MutualSelfAttentionControl(STEP, LAYER)
    editor = MutualSelfAttentionControl(STEP, LAYER)
    regiter_attention_editor_diffusers(model, editor)

    # inference the synthesized image
    image_masactrl = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)[-1:]
    # Save the edited image
    file_name, file_ext = os.path.splitext(os.path.basename(file))
    # image_masactrl.save(f"{output_folder}/final_test_{file_name}.png")  # with attention hijack
    save_image(image_masactrl, f"{output_folder}/final_test_{file_name}.png")  # with attention hijack


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]
Using MutualSelfAttentionControlMaskAuto


DDIM Sampler: 100%|██████████| 50/50 [01:06<00:00,  1.33s/it]


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]
Using MutualSelfAttentionControlMaskAuto


DDIM Sampler: 100%|██████████| 50/50 [01:07<00:00,  1.34s/it]


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]
Using MutualSelfAttentionControlMaskAuto


DDIM Sampler: 100%|██████████| 50/50 [01:07<00:00,  1.35s/it]


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]
Using MutualSelfAttentionControlMaskAuto


DDIM Sampler: 100%|██████████| 50/50 [01:07<00:00,  1.35s/it]


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]
Using MutualSelfAttentionControlMaskAuto


DDIM Sampler: 100%|██████████| 50/50 [01:06<00:00,  1.34s/it]
